In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install gtts pydub audioop-lts
from pydub import AudioSegment
import time
from datetime import date
import os
import copy
import pandas as pd
from gtts import gTTS
from collections import defaultdict
from utils_data import (
    load_raw_data, check_dups, filter_df_to_vocab_of_interest, 
    fill_default_settings
    )
from utils_audio import (
    create_tts_files_for_one_vocab_word, compute_pinyin_and_create_recordings,
    combine_audio_files_and_compute_durations
)
from video_file_configs.a002_second_refresh import dict_recordings
pd.options.display.max_columns = 100
pd.options.display.max_rows = 130


# 0. Define audio

In [3]:
audio_text_lists = [
# Introduction
    [
    """
知识点汇总
1.根据题型及分数安排复习重点
在《艺术概论》考卷上,其题型主要有择题、问答题、作品赏析题、论述题等,
每个题型所占的分值比例各不相同,出题思路也有所差异,因此,考生可以将不同
题型考察的重点对照考试大纲进行标注,然后根据标注的内容认真学习,将重点知
识在脑海中都过一遍,并搭配一定的真题练习进行巩固。
""",

    """
2.根据试题内容选择备考重点
在备考《艺术概论》的选择题时,我们不难发现,无论真题的题型如何变更,
其考察的内容仍然是基础知识。虽然出题方式不同,但考查的知识点难度大同小异。
如考查作品、作家等知识点,这都是各章节中的原题。考生可以用特殊颜色的记号
笔将这些在题中出现的知识点进行标注,然后按照标注有选择的进行背诵,这不仅
降低了我们的备考难度,也能使我们的复习更具备针对性。 
""",],

# Chapter 1
    [
    """
第一章——绪论
【要点】了解本学科的研究对象、学科性质、学科发展及任务、研究方
法等。
    """,

    """
(一)研究对象及学科性质
1.研究对象
艺术概论的研究对象是人类的艺术活动,以及与之相关的概念、范畴、理论和方法等。
艺术有广义与狭义之分。
※广义的艺术包括实用艺术(建筑、园林、工艺美术与现代设计)、造型艺术(绘画、
雕塑、摄影、书法等)、表情艺术(音乐、舞蹈等)、综合艺术(戏剧、戏曲、电影、电视艺
术等)、语言艺术(诗歌、散文、小说等)。
※狭义的艺术专指语言艺术以外的其他艺术种类。语言艺术也称文学,人们有时把文学
与狭义的艺术并称为文艺。
我们这门“艺术概论”课程所使用的是广义的艺术概念。艺术活动是指人类从事的一切
艺术行为。
    """,

    """
2.学科性质
艺术概论是一门研究艺术活动基本规律的学科,是阐述艺术的基本性质、艺术创作与传
播接受系统、艺术种类和特点的科学体系。
人类的艺术活动是伴随着人类社会的产生而产生的。早在数万年前的原始社会,人类就
开始了音乐、舞蹈、绘画等艺术活动。随着社会的不断发展和进步,人类的艺术活动越来越
丰富,随之产生了一系列艺术思想与艺术理论。如西方古希腊的许多著名哲学家、思想家、
艺术家在他们的著作中提出了许多艺术思想,中国自古以来产生的大量的文论、诗论、乐论、
画论等也都是对艺术的研究。这些都为艺术学的产生奠定了基础。
与艺术学联系密切的是美学,于 18 世纪中叶发展成为独立学科的。认为艺术即美,美
学即艺术哲学。
    """,

    """
19 世纪末,德国的康拉德·费德勒(1841-1895)首先提岀了“美学的根本
问题与艺术哲学的根本问题完全不同”的新见解,提岀“美=艺术”的公式不能成立,把美
与艺术、美学和艺术问题的研究区别开来,被称为“艺术学之父”。继费德勒从对象角度提
出艺术学的独立性之后,德国的格罗赛(1862-1927)又着重从方法论上建立艺术科学。他的
《艺术的起源》是艺术社会学的重要著作之一。
进入 20 世纪之后,德国的狄索瓦(1867-1947)、乌提兹(1883-1965)等大力倡导一般
艺术学,出版了一系列学术专著,产生了广泛影响,使艺术学在德国及其他一些国家确立了
学科地位。20 世纪二三十年代,日本、苏联等国相继开展了对艺术学的研究和探讨,我国
也出现了一些艺术学方面的译著和著作,标志着艺术学研究的进一步广泛和深入。
    """,

    """
    (二)学科任务
1.系统地阐述艺术的本质及其特性,确立科学的、进步的艺术观。
艺术观是指对艺术本质、艺术的一般规律及相关原理的观点,艺术观是世界观不可或缺
的组成部分,正确的艺术观需要在先进的、科学的世界观基础上建立。马克思主义的辩证唯
物主义和历史唯物主义世界观是指导我们认识和分析艺术现象的基本岀发点,用马克思主义
基本理论和观点分析艺术现象,总结概括艺术规律,确立科学的、进步的艺术观是艺术概论
学科的基本任务之一。
    """,

    """
2.深入论述艺术与社会历史发展的关系,坚持文艺“以人民为中心”的根本宗旨。
马克思主义认为,人民对美好生活的向往是社会发展的根本动力,艺术活动的源泉是人
民的社会实践,艺术源于生活,人民的社会实践活动是艺术创作取之不尽、用之不竭的源泉。
艺术创作活动要坚持以人民为中心,坚持为人民创作;只有深入实践、深入群众、深入生活、
努力创作出人民群众喜爱的精品力作,才能实现艺术家的价值,才能实现艺术服务社会发展
的价值。
    """,

    """
3.了解艺术活动系统各个环节之间的密切联系,探讨艺术活动规律与特点。
艺术活动是由不同环节和部分构成的,这些环节和部分之间存在着有机联系,艺术理论
研究的任务之一就是通过对艺术现象的分析,科学地认识艺术活动系统的各个组成部分和环
节,辨析艺术系统内部的构成规律和艺术活动的内在规律与特点。
    """,

    """
4.指导人们遵循审美规律和艺术规律进行能动的创作、传播、接受和批评。
艺术概论揭示艺术创作规律和审美接受规律,这些规律来源于艺术实践,又反过来指导
人们的艺术实践,使人们自觉地按照艺术规律进行艺术创作、艺术批评和艺术鉴赏。
    """,

    """
(三)研究方法
1.坚持辩证唯物主义和历史唯物主义的基本原则。
2.采用逻辑的和其他行之有效的科学方法,研究艺术活动的实践与发展。
3.运用艺术学科特有的研究方法,研究艺术领域的各种问题。

绪论部分对艺术概论进行简要介绍,通过这部分的学习,应主要了解艺术学是一门什么
样的学科,根据考试大纲,这部分内容所占分数比例为 1%左右。 第二章——艺术活动
    """,
    ],

# Chapter 2
    [
    """
第二章——艺术活动
【要点】了解艺术活动的构成,理解艺术活动发生和发展的状况,掌握
艺术活动的基本特征和性质,对艺术活动的功能有基本了解。
    """,

    """
(一)艺术活动的构成
根据艺术活动的历史及其当代发展状况,可以将艺术活动视作一个系统,它由四个要素
或环节构成。
    """,

    """
1.客体世界
客体世界即艺术活动所反映和表现的社会生活、自然界与人生,具有审美价值的客体世
界是艺术创作主体观照的主要对象。
赵树理没有对解放初期山西农村生活的亲身体验,就写不出《小二黑结婚》;老舍没有
对老北京风土人情的切身体会和几十年老北京生活的积累,也写不出《骆驼祥子》《四世同
堂》《茶馆》等优秀文学作品;
没有人类在意识生成和完善过程中逐渐形成的对形式美的审美认识,也不会有书法艺术、
舞蹈艺术、绘画艺术、音乐艺术的发展。所以说,客观社会生活和自然界是构成艺术活动的
要素之一。
    """,

    """
2.艺术创作与制作
艺术创作即艺术家主体基于自身的审美经验和审美体验,运用特定的艺术语言和材料,
将其审美意识物化为艺术形象、艺术情境的创造性活动。
例如作家创作小说、诗歌等文学作品时,就是在其自身的审美经验和审美体验基础上,
运用文学艺术语言进行文学艺术形象和意境的创作;画家在进行绘画作品的创作时,也是基
于自身的审美经验和审美体验,运用线条、色彩等绘画艺术语言进行艺术形象或艺术意境的
创作。艺术创作的特点集中体现为“创造性”活动,艺术家的创作是原创的、不可复制的。
艺术制作是艺术生产的另一种表现形态,它更多地体现出以物质性制作为主的特点。艺
术制作与艺术创作紧密联结,有时也具有相对的独立性。
艺术制作与艺术创作相比,具有“以物质性制作为主”的特点。对物质技术的体现多于
艺术创造性。但是,艺术制作在电影、电视、现代音乐作品的制作等领域,艺术制作具有重
要的意义。所以,我们把艺术创作与制作划分为介于客体世界与艺术作品之间的一个艺术活
动环节。
    """,

    """
3.艺术作品
艺术作品即艺术创作和艺术制作的成果,是由艺术创作主体创遊的审美意识物态化的表
现形式。
艺术作品根据其物态化表现形式的不同,可分为文学作品、绘画作品、音乐作品、舞蹈
作品,以及建筑、园林、影视作品等。
    """,

    """
4.艺术传播与接受
艺术传播即借助一定的物质媒介和传播方式,将多种艺术信息传递给接受者的过程。
艺术接受即鉴赏者以艺术作品为主要对象,进行积极能动的消费、欣赏和批评等活动。
艺术接受与艺术传播紧密相连,艺术传播的目的是为了使艺术作品为更多的人接受,同
时艺术接受又以艺术传播为基础。
    """,

    """
(二)艺术活动的发生和发展
1.关于艺术发生的几种学说
艺术活动的发生是指艺术活动的起源。考古发现,人类最初的艺术活动始于上万年前的
冰河期。由于年代的悠久,艺术起源的问题蒙上了一层神秘的色彩。早在人类社会发展的初
期,处于蒙昧时期的人类就开始试图用神话传说来解释文艺的起源。于是,西方出现了文艺
女神缪斯的神话,中国出现了夏禹的儿子启偷记天帝的音乐并带回人间的传说。
当人类进入文明社会以后,产生了许多不同的解释,其中影响较大的主要有以下几种观
点。
    """,

    """
(1)模仿说
模仿说认为艺术起源于人类对于自然或现实生活的模仿。这是一种有关艺术起源的古老
的理论,在古希腊的哲学家中比较流行,德谟克利特亚里士多德等人均持这种观点。模仿说
在西方的影响极其深远。早在两千多年前,古希腊哲学家德谟克利特说:“从蜘蛛我们学会了织布和缝补;从燕
子学会了造房子;从天鹅和黄莺等歌唱的鸟学会了唱歌。”这种关于艺术起源于“模仿”的
理论有一定的合理之处。因为早期的人类艺术,特别是原始艺术中,绝大部分艺术作品都具
有明显的“模仿”特点,如西班牙北部阿尔塔米拉洞穴和法国蒙地亚克的拉斯科洞穴中发现
的约两万年前旧石器时代的壁画,绘有的马、牛、猪、鹿、熊等动物形象和粗糙的人像,都
是对现实生活中动物和人物各种神情姿态的模仿与记录。
模仿说肯定了艺术来源于客观的自然界和社会现实,其中包含着朴素唯物主义的观点,
具有进步的和合理的内容。但是,这种说法只是触及了事物的表面,而没有揭示事物的本质。
    """,

    """
(2)游戏说
游戏说认为艺术活动起源于人类所具有的游戏本能。由于人类具有过剩的精力,并将某
种过剩的精力投人到无功利性的活动中,体现为一种自由的游戏。德国的席勒、谷鲁斯和英
国学者斯宾塞均持这种观点。
人只有在“游戏”时,才能摆脱自然的强迫和理性的强迫,获得真正的自由,只有通过
“游戏”,人才能实现物质与精神、感性与理性的和谐统一。
(3)表现说
表现说认为艺术起源于人类情感表现和交流的需要。持这一观点的有意大利美学家克罗
齐、英国史学家柯林伍德、美国学者苏珊·朗格等。表现说对西方现代社会的艺术活动产生
了巨大的影响。
但是,把艺术的起源归结为“表现”,脱离开人类的社会实践,脱离原始社会生产力低
下的实际情况,仍然是把现象当作本质,把结果当作原因,同样不能科学地阐明艺术的起源
问题。
    """,

    """
(4) 巫术说
巫术说认为艺术起源于原始民族的巫术仪式活动。这是在近现代西方学术界最具影响的
一种理论。这个学说最早由英国学者爱德华·泰勒提出,英国学者詹姆斯·弗雷泽也持这一
观点。
(5)劳动说
劳动说认为艺术产生的根本动为和原因在于人类的以生产劳动为主体的社会物质实践
活动。该学说是对艺术起源最具有影响力的理论之一。俄国普列汉诺夫等人对此进行过阐释。
俄国马克思主义理论家普列汉诺夫在《论艺术》(又称《没有地址的信》)中阐述了艺术
产生于劳动的观点,他指出:人类社会最初的艺术是从劳动中产生的,因为劳动不仅创造了
人,也创造了人类艺术。
劳动说认为,艺术最初是从生产劳动中萌芽和产生的,人类为自身生存和发展而进行的
劳动是艺术产生的根源。劳动说以历史唯物主义理论为基础,肯定了物质生产实践在艺术产
生过程中的决定作用,其理论核心具有一定的科学性。但劳动说忽视了其他社会因素在艺术
起源过程中的作用,仍然不足以解释各种艺术活动的起源。
    """,

    """
(6) 多元说
多元说认为艺术的发生经历了一个以巫术为中介,以劳动为前提得的漫长历史发展过程,
原始人类模仿自然的本能(模仿说)、表现情感的需要(表现说)、游戏的冲动(游戏说)也滲透
其中,尤其是对于原始人来讲更为重要的原始巫术与原始生产劳动在其中发挥了决定性的作
用。艺术的起源应当是多因的而并非是单因的,归根结底,艺术的产生和发展是人类社会实
践活动的必然产物。
    """,

    """
2.艺术活动的发展
(1) 艺术活动发展的根本因素
1在人类社会生活中,经济基础的发展对于意识形态具有较强的制约和影响作用,艺术
活动的发展也要受到经济的制约和影响。
根据马克思主义理论,人类社会结构分为两大部分,其一是经济基础,其二是上层建
筑。经济基础与上层建筑是对立统一的,经济基础决定上层建筑的性质,上层建筑反映经济
基础,并对经济基础具有反作用。经济基础的变革必然带来上层建筑的变革,先进的上层建
筑的确立反过来加速旧的经济基础的摧毁,促进新的经济基础的形成和巩固,这对矛盾的对
立统一是推动社会进步的强大动力。
    """,

    """
2以物质生产为中心的经济活动,是影响艺术活动发展的主要因素和推动艺术活动发展
的根本动力。但经济对艺术的影响一般不是直接的,而是先影响社会,进而作用于艺术的。
主要体现在如下几个方面。
其一,经济基础及其相关的社会发展为艺术提供了新的艺术主题和艺术题材。
其二,经济基础的发展为艺术带来新的艺术观念。
其三,经济基础影响和制约艺术形式的变化。
3经济的兴衰与艺术的兴衰虽然有着密切的联系,但不是机械地互为因果关系的;两者
之间也不总是同步发展,有时呈现为不平衡的发展状态。
例如;中国的魏晋时代是一个政治黑暗、经济上落后、军事上腐败的历史时期,但在艺
术上却产生了曹氏父子、建安七子、陶渊明等人的诗歌,王義之父子的书法,顾恺之的绘画
等光耀千秋的艺术杰作。
    """,

    """
(2) 艺术发展的继承与创新
1继承和创新是促进艺术活动发展的重要因素。艺术活动的发展是有其内在继承性的,
反映着社会意识形态和人们审美观念的连续.性。后一时代的艺术活动通常要在前一时代艺
术成果的基础上演变与发展。
    """,

    """
2艺术的继承性,首先为对本民族艺术传统的吸收和接受,尤其表现在对艺术的内容、
形式、审美观念和创作方法等方面的继承;其次表现在对其他民族和国家优秀文化和艺术成
果的借鉴与吸纳。
艺术的历史继承性还体现在审美观念和创作方法上。在艺术发展中,继承是手段,创新
是目的。继承固然是艺术发展的重要环节,而创新却是艺术发展的关键。
3艺术发展的过程就是一个不断推陈出新的过程。为了创新,就要坚持批判的原则,对
文化遗产去其糟粕,取其精华,同时要倡导在艺术内容、艺术形式、艺术语言、艺术表现手
法等方面的创新,”坚持创造性转化和创新性发展,不断适应新时代人们对于审美文化和艺
术的需求。
    """,

    """
(三)艺术活动的基本特征与性质
1.艺术是一种审美的意识形态
艺术作为一种特珠的精神文化现象,它既具有一般意识形态的特性,又具有自身的审美
特性,是一种审美的意识形态。作为审美的意识形态,艺术与意识形态各部分,以及政治、
科学等均有着密切的联系。
    """,

    """
(1) 艺术与哲学
哲学旨在研究自然界、人类社会和人的思维等领域中带有普遍性的根本规律,在艺术活
动中哲学主要通过美学这一中介对艺术产生影响和指导作用。艺术可以通过审美创造的成果
及其形象思维对哲学研究产生启迪,在一定程度上影响哲学理论的深化与延展。
例如,卡夫卡的小说《变形记》,情节并不复杂,故事讲一个推销员早晨醒来后发现自
己变成了一只甲壳虫,最后悲惨地死去。这篇小说集中体现出卡夫卡的存在主义哲学观念,
主要是孤独感、危机感、“人即非人”等,具有强烈的非理性主义哲理性色彩。又如唐代诗
人陈子昂的《登幽州台歌》:“前不见古人,后不见来者,念天地之悠悠,独怆然而涕下。”
短短几句,既写出诗人怀才不遇、报国无门的思想感情,又蕴藏着诗人感叹生命有限而自然
无穷的深邃人生哲理。
哲学对艺术的影响,更表现在它能起到促进艺术潮流形成的作用。例如,作为现代主义
文艺的重要流派之一的超现实主义,就是以法国唯心主义哲学家柏格森的直觉主义作为理论
基础。他从唯心论和神秘论的立场出发,认为只有人的梦幻世界或直觉领域才能达到绝对真
实,即所谓的“超现实”。
    """,

    """
(2) 艺术与宗教
艺术与宗教在其早期曾融合在一起,在其后的发展过程中也有着极其密切的关系。二者
在认识与掌握世界的方式上有着相似之处。宗教长期运用艺术的方式来进行宣传,客观上促
进了艺术的发展,同时,艺术也在不断影响着宗教。但宗教和艺术在思维方式和终极目的上
的根本差异决定了二者的本质区别。
(3)艺术与道德
道德是指人们在社会生活中形成的伦理观念和行为规范。道德与艺术的联系相当紧密。
一方面,一定社会的伦理道德总要通过艺术的内容和精神得以体现;另一方面,艺术以审美
的方式对道德观念进行批判并加以表现,从而影响人们的道德观念
    """,

    """
(4) 艺术与政治. 政治是经济的集中反映,政治和艺术都是上层建筑和意识形态的重要组成部分,二者是
相互联系和统一的关系。艺术活动会受到政治的制约和影响,政治在一定程度上可以影响和
引导艺术的创作倾向和发展方向,良好的政治环境可以保障艺术得到更快、更健康的发展。
艺术也可以通过自身显现的审美情感和精神倾向对人们的政治思想产生一定的作用,从而影
响政治的方向和发展。
    """,

    """
(5)艺术与经济
艺术与经济始终具有密切的联系。经济是艺术发展的重要支撑和保障,艺术又以审美文
化的提升影响经济的发展。当代文化创意产业是艺术与经济深度融合的体现,既助推创意转
化成产品,生产出价值,又通过艺术市场实现艺术商品的交换,满足消费者的审美娱乐与精
神需求,提高生活品质。文化创意产业和艺术市场成为社会经济的重要组成部分,在促进艺
术生产、创造经济效益的同时,也在承担着创造精神价值、产生社会效益的责任。无论任何
时候,艺术生产均应把社会效益放在首位。
    """,

    """
(6) 艺术与科技
艺术与科技既有密切的关系,也有本质的区别。现代科技为艺术提供了新的物质技术手
段、传播手段和重要载体,并促使新的艺术形式产生和美学观念发生变化。在创新思维、审
美追求等方面,艺术对科技的发展也有重要的影响。
2.艺术活动的基本性质
艺术活动是以特有的艺术语言体系为媒介,以创造形象或意境来满足人类审美需求为目
的的精神文化活动。
3.艺术活动的基本特征
(1)艺术活动的形象性
形象塑造是艺术活动特有的方式,是主体对于客体领悟式的审美创造,其思维方式以感
性为主,重视审美体验而不以理论分析为主。
    """,

    """
(2) 艺术活动的情感性
艺术活动的情感是贯穿于艺术活动的全过程的特殊情感,于人类一般情感既有联系又有
明显区别,是一种无功利的、多样性的、审美性的情感形态。情感在艺术活动动机的生成、
创作与接受过程中均是重要的心里因素,同时也是艺术作品的基本元素。
(3) 艺术活动的审美性
艺术活动区别于其他具有意识形态属性的社会实践活动,具有较强的美感呈现特征。艺术的意识形态特性是隐含在审美特性之中的,它使艺术的审美世界具有了更为广阔和深邃的
内涵。
    """,

    """
(四)艺术活动的基本功能
1.艺术活动的基本功能
(1)审美认知功能,指人通过艺术活动能够获得关于社会、自然和人生的知识和信息。
首先,艺术对于社会、历史、人生具有审美认知功能,因为艺术活动具有反映与创造统
一、再现与表现统一、主体和客体统一等特点,往往能够更加深刻地揭示社会、历史、人生
的真谛和内涵。
其次,对于许多的自然现象,艺术也同样具有审美认知作用,可以帮助人们增长多方面
的科学知识。
(2) 审美教育功能,指人通过艺术活动受到真、善、美的熏陶和感染,思想上受到启迪。
(3)审美娱乐功能,指人在艺术活动中能够获得精神愉悦,使身心得到调节。
(4)审美体验功能,指人在艺术活动中能够浸入丰富的情感体验,使精神与性情获得陶
冶。
如同艺术创作是一种自由自觉的活动,艺术家在创作中处于一种完全忘我的状态,沉醉
其中,并获得极大的满足和快乐一样,艺术欣赏同样也是一种自由自觉的活动。人们欣赏艺
术作品时,读者、观众或听众也同样处于一种忘我的状态,沉醉在艺术作品之中。
以上四种功能是一个有机整体,不可分割。
    """,

    """
2.艺术教育
艺术教育是美育的核心,也是实施美育的主要途径。美育即审美教育,是指运用审美的
方式实施教育,旨在提高人们的审美感受力、审美创造力及审美情趣,以促进其人格的完善
以及全民族整体素质的提升。艺术教育主要通过指导人们进行艺术创作、艺术鉴赏等活动,
实现美育的最终目标。狭义的艺术教育是指培养专业艺术创作者的特定教育。
美育理论体系的建立则是在世界近代史上才开始的,“美育”这个概念,也是直到近代,
才由 18 世纪德国美学家席勒正式提出来的。
席勒是德国古典美学的代表人物之一,其最主要的美学著作是《美育书简》,在这一美
学理论名著中,他不但首次提出了“美育”这一概念,而且系统阐述了他的美育思想。
    """,

    """小结
对于第二章的内容,需要重点了解艺术活动的构成,理解艺术活动发生和发展的状况,
掌握艺术活动的基本特征和性质,还有艺术的基本功能。根据《复习考试大纲》的要求,这
一部分占试卷内容的 17%,是考试重点内容之一。复习时既要重视掌握有关艺术活动的基础
知识和基本概念,又要注意透彻理解和掌握关于艺术活动的构成、艺术活动的基本特征和性
质等基本理论。
    """,
    ],

# Chapter 3
    [
    """第三章——艺术种类
【要点】理解和掌握艺术分类的基本原则和方法,理解各艺术种类的特
性,了解不同艺术种类体裁的区分,熟悉中外艺术史上重要的艺术家和
艺术作品。
(一)艺术分类的主要方法
    """,

    """
1.以艺术作品的存在方式为依据,艺术可以分为时间艺术(音乐、文学等)、空间艺术(绘
画、雕塑等)和时空艺术(戏剧、影视等)。
2. 以对艺术作品的感知方式为依据,艺术可以分为听觉艺术(音
乐等)、视觉艺术(绘画、雕塑等)和视听艺术(戏剧、影视等)。
3.以艺术作品对客体世界的反映方式为依据,可以将艺术分为再现艺术(绘画、雕塑、
小说等)、表现艺术(音乐、舞蹈、建筑等)和再现表现艺术(戏剧、影视等)。
4.以艺术作品的物化形式为依据,艺术可以分为动态艺术(音乐、舞蹈、戏剧、影视等)
和静态艺术(绘画、雕塑、建筑、工艺等)。
5.以艺术形态的物质存在方式与审美意识物态化的内容特征为依据,艺术可以分为造型
艺术、实用艺术、表情艺术、语言艺术(文学)和综合艺术。
    """,

    """
(二)造型艺术
1.绘画艺术
绘画艺术是一门使用一定的物质材料,运用线条、色彩和块面等元素,通过构图、造型
和设色等手段,在二度空间(平面)里创造出静态的视觉形象或情境的艺术。绘画在造型艺术
中处于基础地位。
绘画艺术的种类,根据使用材料的不同,可分为中国画、油画、版画、水彩画、水粉画
等;根据表现对象的不同,可分为肖像画、风俗画、风景画、静物画、历史画等;根据作品
形式的不同,可分为壁画、年画、连环画、宣传画、漫画等。
中国画:简称国画,在世界美术领域中自成体系,独具特色,成为东方绘画体系的主流。
中国画的主要特点是:
    """,

    """
第一、工具材料一般釆用中国特制的毛笔、墨和颜料,在宣纸或绢帛上作作画。
第二、中国画的另一个重要特点,是在构图方法上不受焦点透视的束缚,多采用散点透
视法(即可移动的远近法),使得视野宽广辽阔,构图灵活自由,冲破了时间与空间的局限。
中国画在构图时,还可以将不同时间和不同空间的事物安排在一个画面中。如五代南唐画家
顾阂中的不朽名作《韩熙载夜宴图》,就是以五段连续的画面来构成一幅长卷,韩熙载这个
人物在不同的画面中多次出现。
又如,北宋画家张择端的著名风俗画长卷《清明上河图》,更是以散点透视法把汴河两
岸数十里的繁华景象描绘在一个完整的画面中,以全景方式展现北宋都城汴京从城郊农村到
城内街市的热闹情景。
第三、中国画将绘画与诗文、书法、篆刻四者有机地结合在一起,讲究诗、书、画、印
的结合,形成了中国画独特的内容美和形式美。
第四、从根本上讲,中国画的特点来源于中华民族悠久的传统文化和丰富的美学思想。
中国画的传统画法有工笔画,也有写意画。
    """,

    """
油画:是西方传统绘画的代表,具有极高的表现力,是西洋画的主要画种,对世界绘画
的发展产生了极大影响。油画因使用油质颜料而得名,油画作品是用油质颜料在布、木板或
厚纸板上画成的,其特点是:色彩丰富,具有强烈的色调层次、光线、质感和空间感,能够
充分表现物体的质感,能真实生动地描绘事物,具有很强的艺术表现力。油画的传统是重视
对自然的忠实模仿。此外,油画颜料有较强的覆盖力,易于修改,为画家提供了艺术创作的
便利条件。
意大利文艺复兴初期的著名画家、雕塑家乔托被认为是欧洲绘画之父和现实主义画派的
鼻祖。
文艺复兴时期的“画坛三杰”米开朗琪罗、达·芬奇和拉斐尔,分别创作了《创世记》
《最后的晚餐》《蒙娜丽莎》《西斯廷圣母》等作品。
    """,

    """
17、18 世纪,欧洲绘画进一步摆脱了宗教的束缚,肖像画、风景画、风俗画、静物画、
动物画都获得了极大的发展,涌现出一批杰岀的画家,如荷兰的伦勃朗(代表作品有《夜巡》
《自画像》等)、尼德兰的鲁本斯(代表作品有《阿马松之战》等)、西班牙的委拉斯贵支(代
表作品有《教皇英诺森十世肖像》等)、法国的夏尔丹(代表作品有《饭前祈祷》等),以及
英国的透纳(代表作品有《战舰归航》等)。
    """,

    """18、19世纪涌现出许多美术流派和著名画家,其中有法国新古典主义的代表人物雅克·路
易·大卫(作品有《马拉之死》等),法国浪漫主义美术的代表人物籍里柯(作品有《梅杜
萨之筏》等)和德拉克洛瓦(作品有《自由女神领导人民》等),法国批判现实主义的代表
人物库尔贝(作品有《石工》等)和米勒(作品有《拾穂者》等),法国印象主义美术代表
人物莫奈(作品有《日出·印象》《草垛》等)和雷诺阿(作品有《浴女》《包厢》等),以
及法国后印象主义的代表人物塞尚(作品有《苹果和橘子》等)和高更(作品有《塔希提的
妇女》等)。
    """,

    """
英国、美国、瑞典、丹麦都出现了一批优秀的艺术家,特别是俄国“巡回画派”集中了
一批杰出的画家,其中最著名的是列宾(作品有《伏尔加河上的纤夫》《不期而至》)和苏里
柯夫(作品有《近卫军临刑的早晨》《女贵族莫洛佐娃》)。
版画:特点是采用笔画和刀刻的方法,在不同材料的版面上逬行刻画,然后印制岀多份
原作。由于所用的版面材料和性质不同,版画可分为三大类:木刻和麻胶版画的“凸版”类、
铜版画等的“凹版”类、以及石版画等的“平板”类。“凸版”类是在木板或麻胶版上刻制
而成;“凹版”类则是用酸性液体腐蚀铜版形成各种凹线,从而构成具有独特艺术效果的画
面;“平版”类是在石印术基础上采用特殊药墨制成。
    """,

    """
水彩画:大约产生于 15 世纪末文艺复兴时期,18 世纪起在英国发展为独立画种。水彩
颜料是用胶水调制而成的,颜料透明而单纯,作画时用水溶解颜料于纸上,利用画纸的白地
和水分颜料的相互融合渗透,能表现出一种特殊的透明感觉,以及滋润晕化等效果.水彩画
虽然较难构成鸿篇大作,但使用轻便,能充分发挥水分淋漓酣畅、善于晕化渲染之特点,使
作品具有明快柔和、轻松抒情等独特的艺术魅力。
水粉画:同水彩画一样,都是用水调和颜料而作画的,但水彩颜料非常透明,没有覆盖
力,而水粉颜料则一般不透明,不同程度地含有粉质,具有一定的覆盖力。
除以上画种之外,还有用铅笔、木炭、钢笔作画的素描等。
    """,

    """
2.雕塑艺术
雕塑是一门直接利用物质材料,运用雕刻或塑造的方法,在三度空间(立体)中创造出实
体形象的艺术。雕塑可分为圆雕、浮雕和透雕。
圆雕:又称“浑雕”,是指不附在任何背景上,可以从四面观赏的立体雕塑。圆雕的特
点是:立于空间中的实体形象,在创作时必须考虑它的体积感与厚重感,在塑造形象时还必
须照顾到人们从不同的角度进行观赏。
浮雕:又称“凸雕”,是指在平面上雕出凸起的艺术形象。根据表面凸起程度的不同,
浮雕又分为高浮雕(高低起伏大,凸起程度深)和浅浮雕(高低起伏小,凸起程度浅)。
透雕:又称“镂空雕”,是指在浮雕基础上,保留凸岀的物像部分,而将背景部分进行
局部或全部镂空雕刻。
此外,雕塑还可以从其他方面进行分类,如从制作工艺来看,它可以分为雕和塑两大类。
雕,有石雕、木雕、玉雕等;塑,有泥塑、陶塑等
    """,

    """
3.摄影艺术
摄影艺术是采用摄影技术和手法获取静态图像来反映客体世界,进而表现主体审美情感
的艺术。构图、光线和色调是其主要造型手段。摄影艺术包括肖像摄影、建筑摄影、风光摄
影、舞台摄影等。
肖像摄影:又称人物摄影,是以表现人物形象为主的摄影,包括特写镜头、头像、半身像、全身像和群像等。
建筑摄影:也是摄影艺术的一种,它是摄影师运用一定的摄影技术和手段,专门拍摄精
心挑选出来的建筑物,将摄影美和建筑美结合起来的一种方式。
风光摄影:其关键是将自然美转化为艺术美,它不能停留在仅仅再现自然景物上,而是
必须寓情于景,使作品情景交融、意境盎然,在富有诗情画意的自然风光中,表现艺术家的
思想情感和美学追求。
舞台摄影:是指以舞台演出为拍摄对象的一种摄影艺术形式。它一般需要较高的摄影造
型技术,还需要摄影师对所拍摄的艺术表演有较全面的了解,能充分掌握舞台演出的风格样
式、艺术特征以及演员的表演特色,充分展现舞台艺术的风采和演员高超的表演水平。
    """,

    """
摄影艺术的风格和流派:
绘画主义摄影。从 19 世纪中叶起源于英国,很快传遍世界各国,成为摄影艺术史上最
早形成、影响最广的一个流派,它在创作上追求绘画效果,作品从构图布局到光影调节都有
极严谨的法则,该派曾风行一时。
(1)纪实主义摄影。它至今仍是摄影艺术中最重要的一个流派,该派从照相机能真实
还原客观事物形貌的特点出发,强调摄影的纪实性,注重直接而逼真地再现客观现实生活,
崇尚质朴无华的艺术风格
(2)印象主义摄影。它是美术上印象主义思潮在摄影艺术领域的反映,主张摄影艺术
应当表现摄影者的瞬间印象和独特感受,讲究形式美和装饰性,追求在摄影作品中达到一种
朦胧模糊的画意效果,尤其注重色彩与光线的表现。
(3)超现实主义摄影。它是现代主义摄影流派之一,其美学思想与超现实主义绘画基
本相同,在创作时常利用剪贴和暗房技术为主要的造型手段,采用叠印叠放、多重曝光、怪
诞变形、任意夸张等手法,将“超现实的神秘世界”作为表现对象。除此之外,西方现代派
摄影还有抽象派摄影、前卫派摄影等。
    """,

    """
4.书法艺术
书法艺术是以线条組合变化来表现文字之美的艺术形式,它主要通过用笔用墨、结构章
法、线条组合等方式进行造型和表现主体的审美情操。中国书法主要包
括篆书、隶书、草书、楷书、行书等书体。
书法是建立在汉字基础上的艺术,书法艺术的发展同汉字的发展密
不可分。
书法是以线条的组合、变化来表现文字之美的艺术形式,它主要通
过用笔用墨、结构章法、线条组合等方式进行造型和表现主体的审美情
操。书法是建立在汉字基础上的艺术,书法艺术的发展同汉字的发展密
不可分。
早在三千多年前的殷代,刻在龟甲、兽骨上的“甲骨文”,就是以象
形为基础的汉字,奠定了书法艺术的一些基本要义。
商周至战国时代的“金文”脱胎于“甲骨文”,并且渐趋齐整雄伟。
    """,

    """
秦代统一了文字,由大篆变为小篆,所见书迹有泰山、琅那、碣石、会稽等地的刻石,
以及竹简、刻符等,风格端庄,形体更加匀润流畅。

魏晋时期是书法艺术繁荣发展的时期,楷书、行书、草书等各种书体更加齐备和完善,
并且涌现出王羲之等成就卓著的书法大家,对后世书风产生了深远影响。

唐代是书法艺术的鼎盛时期,各种书体都有了承先启后的发展,尤其是楷书的成就达到
高峰。
唐代书法注重书家感情个性的发挥,追求自由的个性表现,故有“宋人尚意”之说,出
现了苏轼、黄庭坚、米芾、蔡襄四大家,具有各自独特的风貌。

元、明、清书法上承古意,个人风格更加鲜明多样,成为书法艺术发展史上又一繁荣的
时代。
书法主要通过文字字体的用笔用墨、点画结构、结构章法、线条组合等造型美,来表现
人的气质、品格和审美情操。
    """,

    """
(三)实用艺术
1. 设计艺术
设计艺术是工业革命后在国际上兴起的一门交叉性、应用性艺术门类。它围绕着某种制
作或生产目的进行富有审美情趣的创作,主要包括产品设计、环境设计、视觉设计等。
一是产品设计,从家具、服装、餐具等日用品到汽车、飞机、电脑等高新技术产品,都
属于产品设计的范畴。它的突岀特点是将造型艺术与工业产品结合起来,使工业产品艺术化。
二是环境设计,指人类对各种自然环境因素和人工环境因素加以改造和组织,对物质环
境进行空间设计,使之符合人的行为需要和审美需要。
三是视觉设计,指人们为了传递信息或使用标记所进行的视觉形象设计。从狭义上讲,
视觉设计又被称为平面设计,主要包括装帧设计、印刷设计、包装设计、展示陈列设计、视
觉形象设计、广告设计等。
    """,

    """
2.建筑艺术
建筑艺术是一种实用与审美相结合。以形体、线条、色彩、质感、装饰、空间组合等为
艺术语言和表现手段,建构成实体形象的造型与空间艺术。建筑可分为民用建筑、公共建筑、
园林建筑、宗教建筑、纪念性建筑
3.园林艺术
园林艺术利用多种技术手段和艺术手法,将山水、花木、建筑等要素组合成为统一的景
观。园林艺术体现了自然因素和人文因素的有机融.合。园林艺术在广义上是建筑艺术的一
种类型。园林艺术的类型,从世界范围看主要有三种,即欧洲园林、阿拉伯园林和东方园林。
东方园林以中国园林为代表。
东方园林 以中国园林为代表。中国园林的历史悠久,到秦汉时的帝王宫苑已具有很大
规模,形成了以湖水为中心,堆山建岛,修筑宫室的“一池三山”的传统中国园林格局。中
国园林的基本特点是崇尚自然,但又不是对自然的简单模仿,而是对自然的艺术再现。
    """,

    """

欧洲园林 以法国园林为代表。它崇尚“人工美”,要求结构布局均衡匀称、井然有序。
这种园林由规整性的几何图案构成,花坛、道路、水池、草坪和修剪过的矮树等互相配合,
平坦开阔,一览无余。

阿拉伯园林 发端于古代巴比伦和波斯。因为当地干旱缺水,很重视水的利用。波斯园
林就形成了以十字形道路交叉处的水池为中心的格局。
    """,

    """
4 工艺艺术
工艺是一种历史悠久的艺术样式,它既有一定的审美价值,同时又具备一定的实用价值,
与制作或生产密切相关。
工艺直接受到物质材料和生产技术的制约,具有鲜明的时代风格和民族特色。工艺的范
围极其广泛,几乎包括除建筑以外人类所有的日常生活用品的制造工艺。具体来讲,工艺品
主要包括以下三大类:
第一类是经过艺术处理的日常生活实用品,如漂亮的绣花枕套、精致的被面床单、美观
的玻璃器皿等。
第二类是民间工艺美术品,如竹编器件、草编器件、蜡染织物、泥塑、木雕、剪纸等,
它们采用的原材料一般比较低廉,工艺比较简单,价格也比较便宜,既可供使用,又可供观
赏。
第三类是特种工艺美术品,如景泰蓝艺术陶瓷、象牙雕刻、玉雕、金银摆件等,它们采
用的原材料比较珍贵,工艺非常精细,价格也比较昂贵,主要供观赏和珍藏之用,这些特种
工艺品实际上已经不具有实用价值,而是主要具有审美价值和艺术价值了。
    """,

    """
(四)表情艺术
1.音乐艺术
音乐艺术是以人声或乐器声音为材料,通过有组织的乐音在一定时间长度内营造审美情
境的表现性艺术。音乐艺术以旋律、节奏、和声、配器、复调等为基本手段,以抒发人的审
美情感为目标,具有情感表现力和情绪感染力。音乐可分为声乐和器乐。声乐按演唱者可分为男声、女声和童声;按音域可分为高音、中音和低音。根据乐器的不同种类,器乐可分为
弦乐、管乐、弹拨乐、打击乐等。
    """,

    """
2.舞蹈艺术
舞蹈艺术是以人体动作为主要表现手段,运用造型、表情、节奏、空间运动等要素,创
造形象和表现情感的艺术样式。
舞蹈包括民族舞、民间舞、古典舞、芭蕾舞、现代舞等;按出场人物的数量可分为独舞、
双人舞、三人舞、群舞等。
    """,

    """
(五)语言艺术(文学)
语言艺术即文学,包括诗歌、散文、小说、戏剧文学和影视文学等体裁。文学是以语言
作为表达方式的一种特殊艺术形式。文学语言是指加工提炼了的口头语言和书面文字语言。
创作主体通过对语言的审美把握来塑造艺术形象,表现思想感情。
    """,

    """
1.诗歌
诗歌是文学的基本体裁之一,是用有节奏、韵律的语言反映社会生活和表达思想感情的
艺术。
诗歌分类可分为抒情诗(例如唐代诗人张继《枫桥夜泊》)和叙事诗(例如北朝名歌《花
木兰》);又可分为格律诗(莎士比亚 154 首十四行诗)和自由诗(美国诗人惠特曼《草叶集》
还有我国的徐志摩)。
    """,

    """
2.散文
散文是一种自由灵活、不受拘束的文学样式,能够迅速地表现作者的生活感受,真实地
反映社会生活。散文的选材范围广泛,表现手法多样,结构自由,以形散而神不散的艺术特
长来集中而凝练地体现主题思想。
散文的种类丰富,一般可分为抒情散文、叙事散文和议论散文三大类。
抒情散文:注重在叙事写人时表现作者主观的感受与情绪,通过托物言志或借景抒情,
抒发作者微妙复杂的独特情感,将浓郁的思想感情融会在动人的生活画面之中,如现代作家
朱自清的《荷塘月色》、冰心的《往事》等。
叙事散文:包括报告文学、特写、速写、传记文学、游记等,侧重叙述人物、景物或事
件,尤其是现实生活中的真人真事,并且将作者的主观情感蕴藏在对于人物和事件的叙述之
中,如唐代柳宗元的山水游记《小石潭记》、东晋陶渊明的《桃花源记》、现代朱自清的《背
影》等。
议论散文:主要是指杂文,它将政论性和文学性结合在一起,运用形象生动的语言,以
及比喻、反语,乃至幽默、讽刺等手法作为锐利的武器,通过精辟深刻的说理和妙趣横生的
议论,使文章具有以理服人的理论说服力和以情感人的艺术感染力,成为一种相对独立的文
学样式。
    """,

    """
3.小说
小说是一种以叙述故事、塑造人物形象为主的文学体裁,它的特点是在生活素材的基础
上用虚构的方式来再现生活,因此,人物、情节和环境是小说不可缺少的三个基本要素。
人物在小说中是最基本的构成要素之一。它可以详细描绘人物的外貌、举止,也可以表
现人物的对话、行动。可以通过不同的视角,从其他人物的眼中观察和表现这个人物,还可
以把笔触伸入人的内心世界,通过深层心理描写来塑造有血有肉的人物形象。特别是小说可
以把人物放在错综复杂的社会生活中来描写,从而塑造出具有较高审美价值的、复杂丰满的
典型人物。
情节是小说另一个基本的构成要素。传统小说的情节具有完整性、复杂性和多样性。波
澜起伏的情节不仅可以引人入胜,对读者产生强烈的感染力,还是展现人物性格、拓深作品
主题的重要手段。
环境同样是小说一个基本的构成要素,一般是指文艺作品中人物活动于其中的社会环境
和自然环境。
    """,

    """
4.戏剧文学和影视文学
戏剧文学和影视文学主要指戏剧和影视剧的剧本。电影、电视剧本与戏剧剧本一样,也
属于语言艺术,是一种文学体裁,可供读者阅读。影视剧本的特征主要有三点:
一是影视剧本的语言描写具有较强的视觉性和画面感。
二是影视剧本的语言要采用影视蒙太奇的表达方式,按蒙太奇的组接方式进行结构,以利于拍摄和胶片的剪辑组合。
三是人物对话应少而精。
    """,

    """
(六)综合艺术
1.戏剧艺术
戏剧艺术是指以舞台演出为表现方式,由演员扮演角色,为观众现场展现故事情节的艺
术样式。戏剧以演员的动作和声音为基本表现手段,以舞台美术、音响、服装、化妆、道具
等多种技术手段为重要构成要素。
按表现手段的不同,可以分为话剧、歌剧和舞剧;按矛盾冲突的性质的不同,可以分为
悲剧、喜剧和正剧;按其容量大小,可以分为多幕剧和独幕剧。
悲剧代表作:希腊著名悲剧家索福克勒斯的《俄狄浦斯王》,莎士比亚的著名悲剧《哈
姆雷特》,法国作家小仲马《茶花女》
喜剧代表作:莎士比亚《威尼斯商人》《第十二夜》,法国剧作家莫里哀《太太学堂》《唐
璜》、俄国果戈理的《钦差大臣》
    """,

    """
古希腊戏剧、印度梵剧和中国戏曲,被称为三种古老的戏剧艺术。
戏曲是中国传统的戏剧形式,它既具有戏剧的一般特点,又具有独特的表现手段和审美
特征。唱、念、做、打是戏曲主要的表现手段。程式化、虛拟性是戏曲的主要审美特征,体
现出高度的综合性。中国戏曲种类繁多,其中有代表性的剧种主要由京剧、昆曲、评剧、豫
剧、越剧、粤剧、秦腔、川剧和黄梅戏等。代表作有:关汉卿的《窦娥冤》、王实甫的《西
厢记》、汤显祖的《牡丹亭》等。
    """,

    """
2.电影艺术
电影艺术是将艺术与科学结合而成的一门综合艺术,它以画面为基本元素,并与声音和
色彩共同构成电影基本语言和媒介,在银幕上创造直观感性的艺术形象和意境。画面、声音
和蒙太奇是电影的主要艺术语言和表现方法。
电影的主要样式有故事片、纪录片、科教片、动画片等,其中作为电影艺术最主要样式
的故事片又可以进一步划分为多种类型。
蒙太奇是电影美学的重要元素,也是电影的基本表现手段。蒙太奇是影视艺术的一种结
构表现方法,它不仅和镜头与镜头的时空转换与连接有关,也和镜头内部的时空转换与连接
有关,还是画面与画面之间的组合关系。
    """,

    """
3.电视艺术
电视属于大众传播媒介,既有传播新闻信息的功能,也有艺术的功能和娱乐的功能。电
视艺术与电影艺术在审美特征上有很多相似之处,它既是综合艺术,又是现代科技的产物,
同时又具有独特的审美特征。
电视艺术主要是指运用电视手段创作和传播的各种文艺作品,主要包括电视剧、电视文
艺专题节目、音乐电视、电视综艺节目等。
特征:在情节、场景、表演诸方面根据观众的审美心理作特殊的艺术处理,给观众以想
象的空间和介入的机会,尽量使观众对剧中人物产生移情和共鸣,从而达到引人入胜的艺术
效果。电视剧在剧情内容、演员表演、服装道具、环境布置等方面,都要求更加亲切、自然
和生活化,直接和逼真地反映生活。
    """,

    """
4.数字艺术
数字艺术是近几十年来随着数字科技的发展而逐渐形成的一门新型艺术样式。凡是以数
字技术为载体,具有独立审美价值的艺术形式,均可视为数字艺术. 数字艺术是技术与艺术的有机结合,技术是其艺术表现的工具基础,艺术则是其创作成
果的审美呈现。数字艺术在数字影像艺术、虚拟现实设计、数字动画、数字博物馆、网络或
手机游戏设计等领域均显现出广阔的应用价值和发展前景。
    """,

    """
小结
本章内容要求理解和掌握艺术分类的基本原则和方法,理解各个艺术种类的定义及分类,
了解不同艺术种类体裁的区分。这部分内容将在考试试题中占约 18%的比例。 第四章——艺术创作
    """,
    ],

# Chapter 4
    [
    """
第四章——艺术创作
【要点】了解艺术家的含义,了解艺术创作是特殊的精神生产,理
解艺术家的修养和能力,掌握艺术创作的基本过程及其各个环节的主要
特点,理解创作活动中艺术家的心理机制和艺术思维状况。
    """,

    """
(一)艺术创作主体
1.艺术创作主体——艺术家
艺术创作主体即艺术家,是人类审美活动的体验者和实践者,也是审美精神产品的创造
者和生产者。他们不但掌握专门的艺术技能与技巧,具有良好的艺术修养和突出的审美能力,
而且具备独立的人格和丰富的情感。
如屈原能够写出千古传诵的著名诗篇《离骚》,是因为屈原虽然被流放,身处逆境,但
内心依然充满真挚热烈的爱国主义情感,充满为祖国和理想献身的强烈愿望。
诺贝尔文学奖获得者、著名美国作家海明威能够成为世人敬仰的艺术家,也是由于他具
有勇敢、正直、刚强不屈同时又富有爱心的突出个性与品格,他是一个用生命写作的人,他
甚至把艺术创作当作他生命的全部意义。
    """,

    """
2.艺术家的修养和能力
(1)艺术家的修养
先进的世界观和审美观,广博的知识、深邃的思想、丰富的情感和独立的人格。
1、先进的世界观和审美观。世界观是人对整个世界,包括自然现象和社会现象等的基
本观点的总和,它包含社会观、伦理观、审美观、人生观、自然观等。先进的世界观决定着
正确的创作目的与创作动机,影响着作品的格调与品位,影响者艺术家的思想水平和艺术创
作能力的提高与发展。
如罗中立的油画作品《父亲》,透过作品中人物满脸的沧桑和慈祥的目光,我们能够感
受到艺术家对父亲的深沉的敬爱,正是这种浓烈而深沉的情感使得欣赏者站在这幅作品前时,
会产生强烈的心灵震撼。
钱绍武的雕塑作品《李大钊》,以大胆的夸张手法塑造了李大钊无比宽大的双肩和挺阔
的胸膛,突出显示了李大钊“铁肩担道义”、为正义事业英勇献身的高大形象,体现了艺术
家对李大钊的敬仰之情,也反映了艺术家的世界观和人生观。
凡·高的油画作品《农夫的鞋》,展现出来的是一只湿乎乎、沉甸甸、沾满厚厚的泥土、
变了形的鞋,它使人联想到农夫在田野的泥土中辛勤、艰辛、长期单调的劳作,艺术家对农
夫的丰富情感蕴含其中,使人感受到压抑、痛苦,甚至产生呐喊的愿望。
    """,

    """
2、广博的知识。要成为艺术家,首先要在生活和实践中不断地学习、积累各方面的知
识,不断地丰富自己的知识宝库。例如《三国演义》《水浒传》等作品中包含十分丰富的军
事知识,以致很多军事家把它们视为必读的参考书籍。《清明上河图》中包含着众多的社会
信息和风俗人情,成为研究北宋人文历史的重要资料。
3、深邃的思想。艺术生产作为一种特殊的精神生产,除了需要艺术家具有广博的知识、
一定的艺术才能和技巧外,还需要艺术家具有深刻的思想修养。优秀的艺术作品都是有意境
的。意境就是使人超越具体有限的物象产生的对人生、历史和宇宙的哲理性感悟,是艺术作
品中蕴含的思想和哲理。艺术家要创作具有意境的优秀作品,就必须具有深邃的思想。
    """,

    """
4、丰富的情感和独立的人格。要成为艺术家,就首先需要培养对生活和生命的热爱情
感,培养健康的生活情趣、对真善美的热爱以及对假恶丑的憎恨;还要培养自尊、自爱、自
强、有理想、有道德的健康人格。艺术这种精神活动是相对独立于物质基础和其他意识形态
的,它要求艺术家时时刻刻忠实于个人的感情和认识,在艺术创作中传达个人的真情实感,
所以艺术家要有率真、诚实、正直、敢爱敢恨的独立人格。
    """,

    """
(2)艺术家的审美创造能力
敏锐的感知能力,丰富的想象力,精湛的艺术技巧
1.敏锐的感知能力。艺术家必须具备从艺术的视角敏锐地感知自然与社会生活的能力,
敏锐地观察和捕捉艺术创作的素材的能力。感知是创作的前提,艺术家总是能在常人所不在
意的普通的事物中发现生活与自然的内涵,寻找与生命的契合点,做出对生活、自然和生命
的独特思考。
2.丰富的想象力。艺术想象是指艺术家在艺术创作过程中,在头脑中对从生活中得来的
有关的诸多表象进行分解、重组、连接等加工,把实际上并不在一起的事物从观念上组合在
一起,使之成为新的理想化了的艺术意象。明代戏曲大家汤显祖在创作《牡丹亭》时,写到
杜丽娘因情生梦,因梦而病,相思病危之际,禁不住自己泪如泉涌,独自躲进一间堆柴的屋
子里哭泣,这种首先自己感受到的艺术想象是许多成功艺术家共同经历过的。
    """,

    """
3.精湛的艺术技巧。艺术家必须具有专门的艺术技能,熟悉并掌握某一具体艺术种类的
艺术语言和专业技巧。清代著名画家郑板桥曾经把画竹的过程分为“眼中之竹”“胸中之竹”
“手中之竹”。这就是讲,艺术家应当有敏锐的观察力和感受力,这样在观察生活时才会有
异于常人普通感受的“眼中之竹”。而所谓“胸中之竹”,已经是现实中竹的客观形象和艺术
家思想感情主观因素相融合而形成的意象。至于“手中之竹”,则是通过画家的创造性劳动,
运用绘画艺术的技巧和独特的绘画语言,把艺术家头脑中的意象“物化”为艺术作品,创造
出可供其他人欣赏的艺术形象。
    """,

    """
(二)艺术创作过程
艺术创作是一个复杂的过程,可以分为艺术体验、艺术构思和艺术表现三个阶段。这三
个阶段表现为连续的和不可分割的整体。
1.艺术体验
艺术体验,是艺术创作的准备阶段。它是创作主体在审美经验的基础上,充分调动情感、
想象、联想等心理要素,对特定的审美对象进行审视、体味和理解的过程。
艺术体验通常包含三部分:材料的储备和审美经验的积累;审美发现和审美领悟的发生;
创作欲望的萌动及动机的生成。
第一部分,材料的储备和审美经验的积累
材料的储备是指生活素材的积累。艺术创作的基础是对现实生活的认识和经历,艺术活动不仅起源于社会实践,而且艺术活动的本源也是社会实践。没有一定的生活阅历和生活素
材的积累,是不可能有成功的艺术创作活动的。
    """,

    """
审美经验是指在生活中对事物的审美感受、审美体验的积累。伴随着社会实践活动与人
生阅历的积累,艺术家不仅认识、熟悉许多生活现象和事物,积累了许多给他们留下了深刻
印象的生活画面和事物形象,还积累了对不同事物的审美感受经验。
第二部分,审美发现和审美领悟的发生
艺术的体验和审美发现是指为了一定的创作目的而进行的集中的艺术体验,以及在审美
体验中产生审美发现的过程。在这一过程中,艺术家需要摒除杂念,全身心地投入到审美体
验中。
第三部分,创作欲望的萌动及动机的生成
深切的生活体验和丰富的感性积累,不仅为艺术创作奠定了雄厚坚实的基础,而且常常
成为艺术家从事艺术创作时内在的心理动力或诱因,成为一种重要的创作动机。艺术家在观
察生活、思考生活、体验生活的过程中,必然有大量的所见、所闻、所知、所感,它们在艺
术家脑海里积累得越来越多,一旦丰富到呼之欲出的程度,艺术家的创作激情就会喷涌而出,
一发而不可收。
    """,

    """
2.艺术构思
艺术构思是指创作者在审美体验的基础上,以一定的创作动机为引导,通过各种心理活
动和审美思维方式,对原始素材进行加工、提炼、组合,在头脑中形成艺术意向,酝酿、筹
划作品内容与形式的思维过程。
艺术构思的方式主要包括整合、变形、移情、意蕴等。
整合:即调动材料储备和生活积累,根据创作意图将有关的意象进行整合,产生复合意
象。整合的运作方式包括叠加、联结、互渗和互融等。如鲁迅的《阿 Q 正传》发表后,曾经
有不少人对号入座,认为鲁迅是在讽刺他们,事实上阿 Q 这一人物形象不是现实生活中的某
个人,而是叠加了当时许多中国人的一般形象特征和性格特征而构思出来的典型形象。
变形:即根据创作意图,对生活原型的某些特征予以强化、突岀或夸张,而把另一些特
征予以弱化、淹没或缩小等。如小说《西游记》中创造的孙悟空、猪八戒等许多生动的艺术
形象和精彩有趣的故事情节,虽然具有极大的创造性,但归根到底,并不是凭空想象产生的,
而是艺术家原有的生活体验与生活积累的变形。
    """,

    """
移情:即将自己在甲事物中体验过的审美情感移入类似甲的乙事物之中。包括把对人的
情感体验移入花草树木、动物甚至无生命的山水等。
意蕴:即挖掘素材当中具有典型哲理意味的特征,创造意蕴深刻的意境。如影片《城南
旧事》结尾,在“长亭外,古道边”的音乐声中,画面上秋日香山火红的枫叶接连六次化入
又化出,这种艺术构思不仅准确地表达了女主人公小英子离别故土的无限伤感之情,也不禁
使人想起《西厢记》中的名句“晓来谁染霜林醉?总是离人泪”的意境。
影片《林则徐》中,林则徐送别友人后,快步登上山头亭子,只见大江中一叶扁舟正顺
流而下,这一独到的艺术构思,情景交融,感人至深,在银幕上的滔滔江水中寄寓了“孤帆
远影碧空尽,唯见长江天际流”的深邃意境,这一镜头也成为中国电影史上的著名镜头。
    """,

    """
3.艺术表现
艺术表现,是指艺术创作者选择并运用特定的艺术语言,将自己艺术构思中已经基本形
成的艺术意象最终呈现为物态的存在,使之成为具体可感的艺术形象或艺术情境。
(三)艺术创作的心理要素与艺术思维
1.艺术创作的心理要素
(1)审美感知
感觉是指人的感觉器官在对事物个别属性的反映。审美感觉是人在审美活动中的一种特殊感觉。知觉是在感觉基础上对事物予以综合的、整体的把掘,审美知觉则是对具有审美属
性的事物的整体性审示。审美感知包括审美感觉和审美知觉。审美感觉和知觉通常是交织在
一起、共同发挥作用的。
    """,

    """
(2)审美想象
审美想象概念:审美想象是在在感觉、知觉和表象的基础上,经过有目的的组织和改造,
创造新的艺术形象、情境、意境和典型的过程。审美想象以艺术家的审美体验和审美经验为
基础,贯穿于创作活动的全过程。
想象分为再造性想象(知觉想象)和创造性想象,艺术家的创作活动主要依靠创造性想象。
审美想象离不开一系列的审美心理活动;审美想象是积极的、能动的和创造性的;审美
想象要符合生活的逻辑和情感的逻辑。
审美联想是指审美主体由当前所感知的事物引发而想起有关的另一事物的心理活动。想
象和联想有着密切的联系,联想往往能激发艺术家的想象。想象与联想可以相互交叉与融合。
    """,

    """
审美想象特性:
第一,审美想象离不开表象记忆。想象的基础是人的独特心理功能与社会生活积累。
第二,审美想象是积极的、能动的和创造性的。想象的本质特征在于改造表象并创造新
形象,在于创造性,想象的直接结果是创造新的形象,任何想象都是具有创造性的。
第三,审美想象要符合生活的逻辑和情感的逻辑。因为想象虽然是一种心理功能,但必
须有生活的依据和情感的依据,而不是空穴来风、空中楼阁,所以审美想象必须遵循生活的
逻辑与情感的逻辑。
审美联想
审美联想是指审美主体由当前所感知的事物引发而想起有关的另一事物的心理活动:想
象和联想有着密切的联系,联想往往能激发艺术家的想象,想象与联想可以相互交叉与融合。
    """,

    """
(3)审美情感
审美情感是审美主体对客观对象是否符合自己的审美需要所做出的一种心理反应;审美
情感在创作活动中是联系感知、表象、想象、联想、理解、判断等心理因素的中介环节,审
美情感推动艺术创作思维进展的内驱力。
艺术创作主体的审美情感更加丰富、深沉和细腻多层次、多方面的,同时具有很强的个
性色彩。
    """,

    """
(4)审美理解
审美理解是作为艺术创作主体在审美过程中对艺术活动规律的认识、感悟或把握;
审美理解是和审美感知、情感、想象等心理因素相互连接与交织的领悟式的理解;
审美理解是美感深化的表现,能够推动审美想象和联想的展开,并促成艺术意蕴
的升华。
    """,

    """
2.艺术思维
(1)形象思维
形象思维是艺术家在创作活动中从发现和体验生活,到进行艺术构思、形成艺术意象,
再将其物化为艺术形象或艺术情境的整个过程中所采取的一种主要的思维方式,它是以具体
的、整体性的形象为思维材料所进行的复杂心理活动。
形象思维具有具象性、情感性、创造性等特点。具象性是指形象思维始终要以具体可感
的事象或物象作为思维的基本元素,形象思维的情感性是指形象思维过程中渗透着丰富的审
美感情色彩,创造性是指在原有思维基本元素基础上促成新的形象或情境的生成。
    """,

    """
(2)抽象思维
抽象思维是运用逻辑来进行判断、推理和论证的思维方法;
抽象思维与形象思维在艺术家的创作活动中相互作用,互为补充和促进。
(3)灵感
灵感是指艺术家在创作过程中的某一瞬间,突然出现的精神高度亢
奋、思维极为活跃的一种特殊心理现象,呈现出远远超出平常水准的创作能力,使艺术
构思或传达通过顿悟获得飞跃;灵感具有突发性、超长性、易逝性等特征。
灵感与形象思维有一定联系,二者往往交互影响并发挥作用。
    """,

    """
小结
本章主要从“艺术创作主体”“艺术创作过程”“艺术创作的心理要素与艺术思维”三大
方面阐述了有关艺术创作的几个核心问题。这一章的内容是本书的重点内容之一,将在考试
中占约 22%的比例。
本章的基本学习要求如下:了解艺术创作是特殊的精神生产,理解艺术创作主体一艺术
家的含义,理解艺术家的修养和能力,理解艺术创作中主体与客体的关系,掌握艺术创作的
基本过程及其各个环节的主要特点,理解创作活动中艺术家的心理要素和艺术思维状况。 
    """,
    ],

# Chapter 5
    [
    """
第五章——艺术作品
【要点】理解艺术作品的主要构成,掌握艺术作品三个层次的内涵,深
刻理解典型和意境的意义。对于艺术风格、流派和艺术思潮有基本的了
解。
(一)艺术作品的构成
    """,

    """
1.内容与形式的统一
(1)艺术作品的内容,是指经过艺术家体验、加工和创造的人类生活,其中既包含着对
客体世界的能动反映,又凝聚着艺术家对生活的感受和评价,融入艺术家的知、情、意。在
具体艺术作品中,艺术内容主要指题材、主题、人物、环境、情节等诸多要素的总和,题材
和主题是艺术内容的主要要素。
首先,艺术作品的内容来自主客观的统一。其次,艺术作品的内容是艺术家在意识中创
造出来的,具有一种精神性的内涵。第三,艺术作品的内容不是抽象的概括说明,而是题材、
主题、人物、情节等因素融会而成的具体、生动、完整的形象展现。
题材指的是构成作品内容的基本材料,即作品中表现出来的具体的客观生活环境、情节、
人物和情感等。题材是艺术家对客观社会生活的提炼和概括,它来自社会生活,但又经过了
艺术家的选择和加工,成为作品主题得以表现的基础。
主题,又叫主旨、题旨或主题思想。艺术作品的主题是指通过艺术形象所表现和揭示出
来的主要思想内涵。
    """,

    """
(2)艺术作品的形式,是指艺术作品内容的存在方式,亦即作品内部联系和外部表现形
态。内部联系包括内容诸要索之间的相互联系和组织方式,主要指结构;其外部表现形态呈
现为表现内容的方式和手段,包括艺术语言、艺术手法、类型体裁等。
结构,是指艺术作品内在的组织和构造。结构的作用,是通过一定的组织方式和构造安
排,将艺术作品中的各个部分结合成完美统一的整体。
    """,

    """
(3)艺术作品的内容和形式是不可分割的整体。内容与形式的谐统一才能充分体现艺术
魅力。
在艺术创作中,内容常常起到主导的作用,形式的选择应以是否可以适应内容的需要为
原则;但同时,形式又呈现相对的独立性,它不仅可以具有独立的审美价值,而且由于形式
的变化,可以直接影响和制约内容的审美价值的实现。艺术作品内容与形式和谐统一才能充
分体现艺术魅力。
内容对形式的主导和决定作用,还表现在艺术形式总是随着内容的发展变化而发展变化。
随着社会生活的变化,艺术作品的内容与形式也在变化。内容与形式相比较,内容的变化先
于形式,内容变化了,才引起形式的相应变化。
    """,

    """
2.感性与理性的统一
(1)在艺术作品中,感性更多地是指隐含于作品之中的情感因素以及寄寓在形式之中的
感性特色。
(2)在艺术作品中,理性是指通过作品的形象、意境及形式所凸出的理性内涵。
(3)艺术作品是感性与理性的统一。一方面,艺术作品表现为感性与理性的统一;另一
方面,艺术作品还表现为情感因素与理性精神的统一。
    """,

    """
3.再现与表现的统一
(1)再现,是指在艺术创作中将客体世界及人物真实地呈现于作品之中。如《红楼梦》,
再现了清朝末年封建大家族及其整个时代环境的客观社会生活等。
(2)表现,是指在艺术创作中重在表达主体的情感和理想,以及主体对客体世界的思考
和评判。《红楼梦》中不仅对客观世界进行了生动再现,同时也表达了作者曹雪芹对清末社
会现实和《红楼梦》中再现的各种典型人物命运的深刻思考与评判,以及作者的情感与理想。
(3)再现和表现从来都是紧密联系的。再现性艺术作品同样需要融入主体的情感和理想;
表现性作品虽然所表达的主要是心理和情感的真实,但也具有再现性。
    """,

    """
(二)艺术作品的层次
艺术作品分艺术语言、艺术形象和艺术意蕴三个层次。
第一层是艺术语言,它是作品外在的形式结构,是由声音、形体、画面、色彩、线条及
语言文字等所构成的层次。
第二层是艺术形象,它是作品通过艺术语言所表现出来的物态化形式,是艺术家的审美
意象的外在表现形式,不同的艺术作品可以分别具有视觉形象、听觉形象、文学形象等。
第三层是艺术意蕴,优秀的艺术作品往往具有巨大的普遍性和深刻的思想性,具有象征
和寓意,哲理或诗情,这是深藏在艺术作品中的精神内涵层次。
    """,

    """
1.艺术语言
(1)艺术语言是创作主体在特定艺术种类的创作活动中,运用独特的物质材料和媒介,
按照审美法则,进行艺术表现的手段和方式。
各门艺术都有自己独特的艺术语言,例如,建筑艺术的语言包括“面”和“体”。艺术
语言具有两个最基本的功能:其一是塑造艺术形象、传达艺术内容;其二是创造形式美,具
有直接作用于人的感官的审美功能。例如,当我们在欣赏法国 19 世纪著名印象派画家莫奈
的代表作品《青蛙塘》首先注意到的就是它与众不同的线条、色彩和构图。
    """,

    """
在这幅典型的印象派风格绘画作品中,莫奈特意使用了富有节奏感的笔触来表现水面匕
光的波纹,这种笔触所表现岀来的光的波动,突出地给人以波光闪耀的感觉。此外,这幅作
品在构图上也很有特点,画家把水中的游艇精心设计成一种放射的形状,加上一座小桥向画
面外延伸的线条,使人感觉到画面虽小,但这个水上游乐园的面积似乎很宽广,正是这种独
特的线条、色彩和构图,正是这种精心构思、具有独创性的艺术语言,使这幅作品成为世界
名画之一。
    """,

    """
(2)艺术语言是各门类艺术创作的媒介和形式的总称,可具体划分为文学语言、音乐
语言、绘画语言和视听语言等。
(3)艺术语言具有多样性和丰富性,并在艺术发展中不断变化和革新。
艺术语言它是作品外在的形式结构,是由声音、形体、画面、色彩、线条及语言文字等
所构成的层次。
    """,

    """
2.艺术形象
艺术形象是艺术反映人类生活的特殊方式,是通过审美主体
与审美客体的相互交融,由主体创作出来的艺术成果。艺术形象
既包括鲜明生动的人物,也包括其他富有审美特征的意象、情境
和意境密切关联。
例如画家李苦禅在粉碎“四人帮”以后创作的一幅国画《怒
放图》。画的是梅花,但我们从画面上感受到的绝不仅仅是一枝梅
花,画家是透过那繁茂的红梅抒发了自己从十年内乱中解放出来,
对祖国未来充满希望的无比欢欣的心情。在这里,诗人和画家咏
颂、描绘的梅花不只是梅花形象,还德涵若艺术家创造的情境和
意境。
    """,

    """
(二)艺术形象的特征
艺术形象是个性与共性的统一,感性与理性的统一,思想与情感的统一。
我们可以把艺术形象分为视觉形象、听觉形象、文学形象与综合形象。它们既有共性,
又有各自的特性:
视觉形象,是指由人的眼睛直接感受到的艺术形象:视觉形象的构成材料都是空间性的。
听觉形象,是指由人的耳朵直接感受到的艺术形象,听觉形象的构成材料是时间性的。
例如肖邦著名的钢琴曲《一分钟圆舞曲》,是描写一只爱咬着自己的尾巴急速打转玩耍的小
狗。
    """,

    """
文学形象,是指诗歌、散文、小说、报告文学等依靠语言作为媒介来塑造的形象。例如
《红楼梦》中的林黛玉这个人物,在一千个读者心目中,可能有一千个不同的样子。从这个
意义上讲,文学形象为读者提供了更加广阔的想象天地,读者可以在欣赏过程中更加自由地
进行再创造,获得更多的审美快感。
综合形象,是指话剧、戏曲、电影、电视等综合艺术的形象,其中既有视觉形象、听觉
形象,还有文学形象。它们综合成为一个有机的整体,因此,这些门类中的艺术形象可以统
称为综合形象。
    """,

    """
3.艺术意蕴
艺术意蕴是指在艺术作品中蕴含的深层的人生哲理或精神内涵。它具有多义性、不确定
性和深刻性。艺术意蕴是衡量艺术作品境界高低的重要尺度。
例如:米开朗琪罗的四件大理石雕刻,即《晨》《暮》《昼》《夜》。对于这四件雕刻作品
的含义就有多种存在着很大分歧的说法。
艺术意蕴常常超越了作品自身特定的历史内容,具有更加和深刻的思想内涵,它需要欣
赏者用自己的全部心灵去探究和领悟,它也是作品具有不朽的艺术魅力的根本原因。
    """,

    """
1.典型
(1)典型是指艺术作品中具有丰富个性,体现社会历史真实及其本质规律的艺术形象或
形象体系,它包括典型人物、典型事件、典型环境等。
(2)典型是优秀艺术作品的重要表征,尤其是叙事性艺术作品,通常以塑造典型形象作
为自身的重要任务。
创造艺术典型,要求在鲜明生动的个性中体现出普遍的共性;在对现实关系的表现中要
做到真实描写和深刻把握;要真实地再现典型环境中的典型人物;典型形象通常寄寓了艺术
家的审美理想。
任何艺术典型,要真实的再现典型环境和典型人物;要直接或间接体现艺术家的审美理
想。
    """,

    """
2.意境
(1)意境是中国传统古典美学的一个重要范畴。意境是艺术作品中一种情景交融、虚实
相生的境界,是艺术中主客观因素的有机统一意境中既有来自艺术家主观的“情”,又有来
自客观现实升华的“境”,“情”和“境”是有机地融合在一起的,境中有情,情中有境。
(2)艺术意境一般包含以下三个方面的特点:意境是一种若有若无的朦胧美;意境是一
种有限无限的超越美;意境是一种若隐若现的情愫美。
    """,

    """
(三)艺术风格、艺术流派与艺术思潮
1.艺术风格
(1)艺术风格就是艺术家的创作个性与艺术作品的语言、情境交互作用所呈现出的相对
稳定的整体性艺术特色,风格的形成是艺术家创作度候熟的标志,也是作品达到较高艺术水
准的标志。艺术风格既包括艺术家个人的风格,也包括流派风格、时代风格和民族风格等。
(2)艺术风格的特性
1继承与独创;2稳定与渐变;3多样与同一。
    """,

    """
2.艺术流派
(1)艺术流派是指思想倾向、审美观念、艺术趣味、创作风格相源或相似的一些艺术家
所形成的艺术派别。
形成艺术流派必须具备下面两个基本条件:一是有一些具有一定影响的、或大或小的艺
术家群;二是这些艺术家生活在同一个社会历史阶段,容易产生对社会生活、艺术创造基本
相同或相近的看法和理解,即他们彼此有着相近的思想倾向、审美主张(包括审美趣味),
以至在艺术创造的题材处理、表现手段和方法、艺术风格等诸方面有相似之处。
中外艺术史上涌现了许许多多的艺术流派,形成艺术流派的原因也各种各样,概括起来
讲,艺术流派的形成大致有三种情况:
第一种是自觉形成的,有一定的组织形式和共同的艺术主张;
第二种是不自觉形成的,主要因为创作风格、类型的相近而呈现流派特征;
第三种由他人将其类分和界定为特定流派
    """,

    """
3.艺术思潮
艺术思潮是指在一定社会条件下,由于受到某种社会思潮和哲学思潮的影响,艺术领域
所出现的具有较大影响的思想潮流和创作倾向。
艺术思潮也有它的生长周期。一般说来,艺术思潮总是伴随着社会思潮和哲学思潮而兴
起、生长、衰落,而且艺术思潮也和不同的国家、民族和社会制度有着密切的关系。因此,
艺术思潮的生成、鼎盛与衰落是一种复杂的艺术现象。
    """,

    """
4.艺术风格、艺术流派与艺术思潮的关系
艺术流派是艺术风格相近或相似的创作群体的表现形式。艺术思潮常常包容一个时期内
多个艺术门类中多个流派的艺术主张。
艺术思潮与艺术流派之间有着密切的关系,但又存在着明显的区别。一般来讲,艺术风
格是创作主体独特个性的表现,艺术流派则是风格相近或相似的创作群体的表现形式,而艺
术思潮常常包容一个时期内多个艺术门类中多个流派的艺术主张。
艺术风格、艺术流派与艺术思潮是艺术史上存在的复杂现象,它们与社会思潮、哲学思
潮有紧密的关系,但又绝不是一种简单的对应与图解,因为艺术发展有其自身的规律。
    """,

    """
小结
这一章内容主要对“艺术作品”进行了介绍和分析,其中包括艺术作品的概念与构成,
艺术作品的层次,艺术作品中的典型与意境,艺术的风格、流派与艺术思潮四个方面。学习
这些知识要注意看(听)作品。同时,也要积极调动自己的有关艺术作品的知识积累,融会
贯通地领会有关知识和理论内容。该部分内容将在考试试题中占约 22%的比例。
    """,
    ],

# Chapter 6
    [
    """
第六章——艺术接受
【要点】了解艺术传播的含义,理解艺术鉴赏的意义,掌握艺术鉴赏的
过程,理解艺术批评的内涵与功能。
(一)艺术传播
1.艺术传播指的是艺术信息的传递和交流,是艺术信息在社会系统中的运行。艺术传播
具有公共化与个人化相互交融的特点。它借助一定的物质媒介和传播方式,将艺术信息传递
给艺术接受者,使艺术得到扩展。
    """,

    """
2.构成艺术传播的五个要素是艺术传播主体、艺术传播内容、传播媒介、受传者和传播
效果。
艺术传播主体即艺术传播者,是指艺术传播活动中控制与发送艺术信息的人或机构。
艺术传播内容是指通过传播媒介传送的艺术信息。传播媒介是用来承载并传递艺术信息
的载体和渠道。受传者是指艺术传播活动中接收到艺术信息的受众。传播效果是指艺术信息
在传播活动中产生的效应及其对受传者的影响程度。
艺术传播的方式有很多种。历史上先后出现口头传播、文字传播、印刷传播、电子传播
和网络传播等方式。在现代社会中艺术传播的主要方式有:现场传播方式、展览性传播方式
和大众传播方式。
    """,

    """
(1) 现场表演传播方式,是指传播者与受传者面对面进行艺术信息传播的方式,具有沟
通直接、手段多样、反馈及时、互动性强等特点。
(2)展览性传播方式,是指在一定的场所陈列艺术作品,供观众直接接受艺术信息的传
播方式。
(3)大众传播方式,是指专业化的媒介组织运用先进的传播技术进行大规模艺术信息传
播活动的方式,主要包括报刊、广播、影视、数字媒介等传播方式。
    """,

    """
(二)艺术鉴赏
1.艺术鉴赏的性质
艺术鉴赏是一种以艺术作品为对象,以受众为主体的欣赏活动,是接受者在审美经验的
基础上对艺术作品的价值、属性的主动选择、吸纳和扬弃。它是一种积极能动的再创造活动。
艺术鉴赏作为一种审美再创造活动,主要体现在以下几个方面:
第一,艺术家创作出来的艺术品,必须通过鉴赏主体的审美再创造活动,才能真正实现
它的社会意义和美学价值。
    """,

    """
第二,鉴赏主体在进行鉴赏活动时,并不是被动和消极地接受,而是积极主动地进行着
审美再创造。这是因为任何艺术作品不管表现得如何全面、生动、具体,总会有许多“不确
定性”与“空白”,需要鉴赏者通过想象、联想等多种心理功能去丰富和补充。
第三,从最根本的意义上讲,艺术鉴赏同艺术创作一样,也是人类自身主体力量在审美
活动中的自我肯定与自我实现。艺术生产作为一种特殊的精神生产,决定了艺术必须具有主
体性的特征,这种特性不仅表现在艺术创作和艺术作品之中,还同样表现在艺术鉴赏之中。
它集中表现在鉴赏主体总是要根据自己的生活经验、艺术修养、兴趣爱好、思想情感理想,
对作品中的艺术形象进行加工改造、补充丰富,进行审美的再创造。
    """,

    """
2.艺术鉴赏的主体能动性
(1)对艺术作品审美娱乐属性的享用。
一般人接近艺术作品、鉴赏艺术作品的最直接目的是获得审美享受,鉴赏主体在艺术接
受活动中具有很强的体验意味。人们把鉴赏艺术作品的活动当作一种娱乐活动,一方面鉴赏
者以自己内在和既有的经验体味作品中的意蕴,形成观照客体对象时的心灵体验;另一方面,
又以自己情感的投入,感受心灵和情感的激荡。
    """,

    """
(2)对艺术作品审美认知属性的认知。
艺术作品的认知作用也必须通过鉴赏主体得以实现鉴赏者通过对艺术作品的感知和理
解,一方面认识艺术作品的形式和内容,了解艺术专业知识,提高对艺术作品的鉴赏能力;
另一方面认识艺术作品中表现的社会知识、自然知识等。
(3)对艺术作品文化价值的阐释。
鉴赏者只有具备相应的文化知识修养,才能在艺术鉴赏活动中认识艺术作品的文化价值。
一个没有历史知识和建筑艺术知识的人,到故宫看到的只是一群房屋而已,不可能从中看岀
它的文化价值和建筑艺术价值。
    """,

    """
(3) 对艺术作品形象或意境的再创造。
这一点更有赖于鉴赏者的生活阅历、审美经验的积累。
艺术修养与鉴赏力的培养与提高,主要有以下几个途径:
第一,大量鉴赏优秀作品。艺术鉴赏的实践经验非常重要。多听音乐就能培养和提高乐
感;多看绘画就能训练和发展眼睛的形式感;文学作品读得多了,读得熟了,也就有了比较,
有了鉴别和欣赏力。大量地、经常地鉴赏优秀的艺术作品,有助于人们艺术修养与鉴赏力的
培养与提高。
第二,熟悉和掌握艺术的基本知识和规律。艺术修养包括对一般艺术理论和艺术史的初
步了解,也包括对各个艺术门类和体裁的艺术特征、美学特性和艺术语言的熟悉和了解。
第三,不断积累社会、历史、哲学等方面的文化科学知识。文化知识水平对艺术鉴赏也
有很大影响,广泛的历史、文化知识十分重要。
第四,在生活经验与生活阅历中积累审美体验。艺术创作离不开社会生活,艺术鉴赏也
同样离不开社会生活。鉴赏主体总是在自己生活经验的基础上去感受、体验和理解艺术作品
的。鉴赏者的生活经验越丰富、越深刻,越有助于对艺术作品的审美欣赏。
第五,美育与艺术教育在培养和提高艺术鉴赏力方面,具有特别重要的地位与作用。美
育与艺术教育作为一个独立或专门的领域,就是要通过培养与提高人们敏锐的感知力、丰富
的想象力和审美的理解力,从而使人们形成健全的审美心理结构。
    """,

    """
3.艺术鉴赏的过程
(1)审美期待
审美期待是艺术鉴赏的准备阶段,指接受主体在欣赏之前或欣赏过程之中,心理上往往
会有一个既成的结构图式,它使接受者具有审美需求,并希冀在欣赏中得到满足。
审美期待可分为形式期待、意象期待和意蕴期待。形式期待由艺术作品的类型或形式特
征而引发的接受者的期待;意象期待指由作品中特定的意象或形象而引发的接受者的期待;
意蕴期待指由作品呈现的深层的审美意蕴、人生哲理和情感境界而引发的接受者的期待指向。
    """,

    """
(2)鉴赏流程
鉴赏流程包括艺术的直觉与感知、体验与想象、理解与创造。
1.艺术直觉与审美感知艺术直觉是指人们在审美活动中对审美对象具有一种不假思索
而即刻把握与领悟的能力。审美感知是指人们在注意审美对象形式特点的同时,也已开始关
注审美对象的意义、鉴赏活动往往是在直觉与感知的心理基础上开始的,它将使鉴赏者完成
作品形式美的注意
2.体验与想象。在鉴赏过程中,主体以自身审美经验为基础,潜入作品情境之中进行审
美体验,不断推进与作品中所包含情感的交流与融合。同时由于审美想象和联想的展开,鉴
赏者可以与作品或艺术家进行交流,洞察其深层意蕴,并逐渐生成审美愉悦。
3.理解与创造。理解既包括对于作品的形象、情境、形式、语言的审美认知,也包括对
于作品整体价值的追寻。艺术鉴赏的结果是接要者审美再创造的完成。鉴赏者对于作品中形
象、情境、典型和意境的补充与完善,正是审美再创造的结晶。
    """,

    """
(3)审美效应
审美效应主要有以下三种表现:
1.共鸣。共鸣是指在鉴赏过程中,鉴赏者被作品中的思想情感理想愿望及人物命运所打
动,从而形成的一种强烈的心灵感应状态。不同时代、阶级、民族的鉴赏者,在鉴赏同一部
作品时可能会产生相同或相近的审美感受,也可称作共鸣。
2.领悟。领悟是指接受者在鉴赏艺术作品时由此引发的对于世界奥秘的洞悉、人生真谛
的彻悟以及精神境界的升华。这是一种更高层次的审美效应。
3.净化。净化是指接受者通过对艺术作品的鉴赏和共鸣的产生使情感得到陶冶,精神得
到调节,人格得到提升的状态。
    """,

    """
(三)艺术批评
1.艺术批评的性质
艺术批评是根据一定的思想立场和美学原则、理论体系,对以艺术作品为中心的一切艺
术现象进行理性分析和审美判断的创造性文化活动。
2.艺术批评的类型
在艺术批评史上出现过多种批评类型,主要都是围绕“客体世界“艺术家”“艺术作品”
“读者(接受者)”等艺术活动要素进行的。
    """,

    """
(1)社会历史批评
社会历史批评主要对艺术作品所展现的人类历史与社会生活状况及其内涵加以剖析和
评价,认为艺术作品必然自觉或不自觉地打上特定社会历史环境的印记,主张艺术作品价值
的高低取决于它对社会历史生活的深刻反映程度以及对社会生活的影响力:如 19 世纪法国
丹纳在其《艺术哲学》中从种族、环境和时代三要素出发,进行艺术批评。
(2)心理批评
心理批判强调从艺术活动主体的心理及其艺术作品中表现的精神世界切入,对艺术家创
作的过程中的心理状态和作品中人物的心理活动展开分析与评判。如 20 世纪奥地利心理学
家弗洛伊德运用精神分析方法来研究艺术现象和艺术作品。
    """,

    """
(3)文本批评
文本批评认为艺术作品本身是一个自足的整体,其内容与形式的意蕴需要从文本中寻求;
注重艺术作品的文本语言和物质媒介,通过对作品本体诸要素的分析来呈现艺术作品的价值。
如 20 世纪西方符号学强调对艺术作品本身或艺术符号进行研究。
(4)接受批评
接受批评模式强调从接受者的角度研究艺术活动,认为受众的接受才是艺术研究的中心
所在。突出了接受者在鉴赏过程中的能动性和创造性。如 20 世纪 60 年代出现在德国的以尧
斯和伊塞尔为代表的接受美学强调从接受者的角度研究艺术作品和艺术史。
    """,

    """
3.艺术批评功能
(1)通过对作品的分析和阐释,评判其审美价值。艺术批评致力于对艺术作品内容和形
式的全面剖析,对其美学价值、历史价值、文化价值和社会意义予以评价并做出理论判断,
确认其在当代以及艺术史中的地位。
(2)通过将批评的信息反馈给艺术家,对创作产生影响。艺术批评将各界对艺术活动及
其作品的评价信息予以传播,使艺术家得到及时反馈,帮助艺术家深化对其创作能力的认识
与理解,博采众长,提高创作水平。
    """,

    """
(3)通过批评的开展,对艺术接受者的鉴赏活动及其消费予以影响和指导。艺术批评可
以引领受众欣赏艺术作品,深化欣赏者对艺术作品精神内涵与艺术形式美感的理解,提升其
艺术价值判断力和审美鉴赏力。同时在引导艺术市场运行及受众艺术消费等方面发挥积极作
用。
(4)通过艺术批评,协调艺术与意识形态等其他文化领域的关系,促进社会文化的发展。
艺术批评通过对多元文化现象及形态的介入将审美意识渗透于其他文化领域,推动艺术创新
与相关领域的深度融合,提高大众的综合文化素质,促进社会文化的全面发展。
    """,

    """
小结
这一部分从艺术传播、艺术鉴赏、艺术批评三个方面阐述了有关艺术接受的知识与基本
理论。学习这部分内容,要注意结合艺术传播的实际,结合自己鉴赏艺术作品的体会来理解。
同时,还要补充自己没有的鉴赏实践活动,如艺术批评。此外,运用所学知识和理论进行艺
术鉴赏,也是全面理解鉴赏理论、巩固鉴赏知识的有效途径之一。这一部分内容将在考试试
题中占 20%。
    """,
    ],
]

# 2. Compute and save recordings

In [5]:
audio_final_path = 'audio_files/products/aji/20250923/'
if not os.path.exists(audio_final_path):
    os.mkdir(audio_final_path)
print(audio_final_path)

entire_start_time = time.time()
for chapter_i, chapter_audio_list in enumerate(audio_text_lists):
    for chapter_audio_i, audio_text in enumerate(chapter_audio_list):
        audio_file_name = f"{audio_final_path}{chapter_i}_{chapter_audio_i}.mp3"
        if not os.path.exists(audio_file_name):
            current_start_time = time.time()
            try:
                gTTS(audio_text, lang='zh').save(audio_file_name)
            except:
                print(f"!!!!!!! FAILURE, wait 52 seconds, {chapter_i}, {chapter_audio_i} !!!!!!!\n{audio_text}")
                time.sleep(52)
                try:
                    gTTS(audio_text, lang='zh').save(audio_file_name)
                except:
                    print(f"!!!!!!! FAILURE, wait 258 seconds, {chapter_i}, {chapter_audio_i} !!!!!!!\n{audio_text}")
                    time.sleep(258)
                    gTTS(audio_text, lang='zh').save(audio_file_name)
            print(f"{(time.time()-entire_start_time):5.1f}s, {(time.time()-current_start_time):5.2f}s, {chapter_i}/{len(audio_text_lists)}, {chapter_audio_i}/{len(chapter_audio_list)}")
        else:
            print(f"{(time.time()-entire_start_time):5.1f}s, ALREADY COMPUTED, {chapter_i}/{len(audio_text_lists)}, {chapter_audio_i}/{len(chapter_audio_list)}")

audio_files/products/aji/20250923/
  0.0s, ALREADY COMPUTED, 0/7, 0/2
  0.0s, ALREADY COMPUTED, 0/7, 1/2
  0.0s, ALREADY COMPUTED, 1/7, 0/9
  0.0s, ALREADY COMPUTED, 1/7, 1/9
  0.0s, ALREADY COMPUTED, 1/7, 2/9
  0.0s, ALREADY COMPUTED, 1/7, 3/9
  0.0s, ALREADY COMPUTED, 1/7, 4/9
  0.0s, ALREADY COMPUTED, 1/7, 5/9
  0.0s, ALREADY COMPUTED, 1/7, 6/9
  0.0s, ALREADY COMPUTED, 1/7, 7/9
  0.0s, ALREADY COMPUTED, 1/7, 8/9
  0.0s, ALREADY COMPUTED, 2/7, 0/25
  0.0s, ALREADY COMPUTED, 2/7, 1/25
  0.0s, ALREADY COMPUTED, 2/7, 2/25
  0.0s, ALREADY COMPUTED, 2/7, 3/25
  0.0s, ALREADY COMPUTED, 2/7, 4/25
  0.0s, ALREADY COMPUTED, 2/7, 5/25
  0.0s, ALREADY COMPUTED, 2/7, 6/25
  0.0s, ALREADY COMPUTED, 2/7, 7/25
  0.0s, ALREADY COMPUTED, 2/7, 8/25
  0.0s, ALREADY COMPUTED, 2/7, 9/25
  0.0s, ALREADY COMPUTED, 2/7, 10/25
  0.0s, ALREADY COMPUTED, 2/7, 11/25
  0.0s, ALREADY COMPUTED, 2/7, 12/25
  0.0s, ALREADY COMPUTED, 2/7, 13/25
  0.0s, ALREADY COMPUTED, 2/7, 14/25
  0.0s, ALREADY COMPUTED, 2/7, 15/2

# 3. Make whole recordings per chapter

In [8]:
entire_start_time = time.time()
for chapter_i, chapter_audio_list in enumerate(audio_text_lists):
    concat_file_name = f"{audio_final_path}艺术概论资料汇总_chapter{chapter_i}.mp3"
    if not os.path.exists(concat_file_name):
        # Construct list of individual audio files
        audio_files_this_chapter = []
        for chapter_audio_i, audio_text in enumerate(chapter_audio_list):
            audio_file_name = f"{audio_final_path}{chapter_i}_{chapter_audio_i}.mp3"
            audio_files_this_chapter.append(AudioSegment.from_mp3(audio_file_name))

        # Concat audio files for this chapter
        concat_file_name = f"{audio_final_path}艺术概论资料汇总_chapter{chapter_i}.mp3"
        audio_concat = audio_files_this_chapter[0]
        for audio in audio_files_this_chapter[1:]:
            audio_concat += audio

        # Export audio
        audio_concat.export(concat_file_name, format="mp3")
        print(f"{(time.time()-entire_start_time):.1f}s, {concat_file_name}")
    else:
        print(f"ALREADY COMPUTED, {concat_file_name}")

ALREADY COMPUTED, audio_files/products/aji/20250923/艺术概论资料汇总_chapter0.mp3
ALREADY COMPUTED, audio_files/products/aji/20250923/艺术概论资料汇总_chapter1.mp3
ALREADY COMPUTED, audio_files/products/aji/20250923/艺术概论资料汇总_chapter2.mp3
ALREADY COMPUTED, audio_files/products/aji/20250923/艺术概论资料汇总_chapter3.mp3
5.9s, audio_files/products/aji/20250923/艺术概论资料汇总_chapter4.mp3
11.2s, audio_files/products/aji/20250923/艺术概论资料汇总_chapter5.mp3
15.9s, audio_files/products/aji/20250923/艺术概论资料汇总_chapter6.mp3
